<a href="https://colab.research.google.com/github/JuanCSFC/PRUEBA-1goqy3u/blob/main/OMR_puesto_todo_junto_2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -y >/dev/null 2>&1
!apt-get install -y lilypond >/dev/null 2>&1
!pip install music21 >/dev/null 2>&1

---
# Aquí se intenta  (éxito) "liberar" un musicxml de los saltos de sistema que traía, y asignarle dinámicamente los que nosostros consideremos; generando nuevos musicxml, y los svg de cada una de sus páginas

## Primer intento:

### Lee un musicxml, lo modifica, alterando el número de compases por pentagrama; retorna 3 musicxml nuevos

Sin embargo, no elimina los que ya había, haciendo que haya pentagramas de muy pocos compases

In [5]:
!pip install music21 verovio
from music21 import converter, layout, stream

# Parámetros
INPUT = 'cancion entera.musicxml'
BASIS = 4          # nº de compases por sistema "normal" (ajústalo a tu gusto)
VARIANTS = [0, 2, 4]  # normal, +2, +4

sc = converter.parse(INPUT)
sc.definesExplicitSystemBreaks = True  # ayuda a que se respeten los saltos al exportar [3](https://music21.org/music21docs/moduleReference/moduleLayout.html)

meas_first_part = list(sc.parts[0].getElementsByClass(stream.Measure))

def make_variant(n_bars_per_system: int, out_fp: str):
    # Quita posibles SystemLayout previos (opcional)
    for p in sc.parts:
        for m in p.recurse().getElementsByClass(stream.Measure):
            for sl in list(m.getElementsByClass(layout.SystemLayout)):
                m.remove(sl)

    # Inserta un salto de sistema al inicio de cada bloque de n compases
    for i, m in enumerate(meas_first_part, start=1):
        if i != 1 and (i-1) % n_bars_per_system == 0:
            # inserta en offset 0 de esa medida
            m.insert(0, layout.SystemLayout(isNew=True))  # genera <print new-system="yes"/> [3](https://music21.org/music21docs/moduleReference/moduleLayout.html)
    sc.write('musicxml', fp=out_fp)

# Genera normal (=BASIS), +2, +4
for extra in VARIANTS:
    make_variant(BASIS + extra, f'cast_{BASIS+extra}.musicxml')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 7.8 MB/s eta 0:00:00


### Este código es la continuación del anterior. A partir de dichos musicxml, genera un SVG de cada página

In [6]:
import json
from verovio import toolkit

def render_svg(mxml_path, out_prefix, scale=40, page_w=2100, page_h=2970):
    tk = toolkit()
    opts = {
        "inputFrom": "xml",      # MusicXML
        "breaks": "line",        # respeta line breaks codificados, paginado automático [2](https://book.verovio.org/toolkit-reference/toolkit-options.html)
        "scale": scale,
        "pageWidth": page_w,
        "pageHeight": page_h,
        # Ajustes útiles cuando metes +2/+4 compases por sistema:
        "spacingSystem": 12,     # espacio entre sistemas (MEI units) [4](https://book.verovio.org/advanced-topics/layout-options.html)
        "spacingStaff": 12,      # espacio entre pentagramas dentro del sistema [4](https://book.verovio.org/advanced-topics/layout-options.html)
        # Para “uniformar” la anchura de compases:
        # "spacingLinear": 1.0,  # si subes linear y llevas nonLinear hacia 1.0, los compases tienden a igualarse
        # "spacingNonLinear": 0.9
    }
    tk.setOptions(json.dumps(opts))
    with open(mxml_path, "r", encoding="utf-8") as f:
        tk.loadData(f.read())
    pages = tk.getPageCount()
    for p in range(1, pages+1):
        svg = tk.renderToSVG(p)
        with open(f"{out_prefix}_p{p}.svg", "w", encoding="utf-8") as out:
            out.write(svg)

# Renderiza las tres variantes
for n in [BASIS, BASIS+2, BASIS+4]:
    render_svg(f"cast_{n}.musicxml", f"cast_{n}")

## segundo intento:

se pretende haer que se eliminen los antigus saltos de sistema, permitiendo controlar el número de compases por pentagrama, pero sin mantener los antiguos saltos

### Limpia el musicxml original de saltos

In [8]:
from music21 import converter, layout, stream

sc = converter.parse('/content/cancion entera.musicxml')

# Elimina TODOS los SystemLayout/PageLayout con isNew=True (saltos importados)
for p in sc.parts:
    # Recorre medidas por si los layouts están anclados a medidas
    for m in p.recurse().getElementsByClass(stream.Measure):
        # System breaks
        for sl in list(m.getElementsByClass(layout.SystemLayout)):
            # si quieres ser restrictivo, comprueba sl.isNew; si no, elimínalos todos:
            m.remove(sl)
        # Page breaks
        for pl in list(m.getElementsByClass(layout.PageLayout)):
            m.remove(pl)

# (Opcional) también puedes limpiar layouts a nivel de score
for sl in list(sc.recurse().getElementsByClass(layout.SystemLayout)):
    sc.remove(sl)
for pl in list(sc.recurse().getElementsByClass(layout.PageLayout)):
    sc.remove(pl)

# Ahora el score está "sin saltos".
# Si quieres exportar ya limpio:
sc.write('musicxml', fp='limpio.musicxml')

PosixPath('/content/limpio.musicxml')

### introduce el número de saltos que deseamos y genera los nuevos musicxml

In [9]:
from music21 import layout

sc.definesExplicitSystemBreaks = True  # ayuda a que los lectores respeten <print new-system> [3](https://music21.org/music21docs/moduleReference/moduleLayout.html)

# Referencia de compases con la primera parte
measures = list(sc.parts[0].getElementsByClass(stream.Measure))

def imponer_compases_por_sistema(n, score, measures_ref):
    # Limpia posibles SystemLayout anteriores (por si re-ejecutas)
    for p in score.parts:
        for m in p.recurse().getElementsByClass(stream.Measure):
            for sl in list(m.getElementsByClass(layout.SystemLayout)):
                m.remove(sl)
    # Inserta saltos cada n compases (comienza nuevo sistema en 1+n, 1+2n, ...)
    for i, m in enumerate(measures_ref, start=1):
        if i != 1 and (i-1) % n == 0:
            m.insert(0, layout.SystemLayout(isNew=True))

# “Normal” (p.ej., 4 por sistema), “+2” y “+4”:
BASIS = 4
for extra in [0, 2, 4]:
    imponer_compases_por_sistema(BASIS+extra, sc, measures)
    sc.write('musicxml', fp=f'cast_{BASIS+extra}.musicxml')

### a partir de dichos musicxml, genera los svg de cada página

In [10]:
import json
from verovio import toolkit

def render_verovio(mxml_path, out_prefix, scale=40, page_w=2100, page_h=2970):
    tk = toolkit()
    tk.setOptions(json.dumps({
        "inputFrom": "xml",  # MusicXML
        "breaks": "line",    # respeta saltos de sistema codificados, autopagina [2](https://book.verovio.org/toolkit-reference/toolkit-options.html)
        "scale": scale,
        "pageWidth": page_w,
        "pageHeight": page_h,
        "spacingSystem": 12, # separa sistemas (ajusta según necesites) [4](https://book.verovio.org/advanced-topics/layout-options.html)
        "spacingStaff": 12   # separa pentagramas dentro del sistema [4](https://book.verovio.org/advanced-topics/layout-options.html)
    }))
    with open(mxml_path, 'r', encoding='utf-8') as f:
        tk.loadData(f.read())
    for p in range(1, tk.getPageCount()+1):
        svg = tk.renderToSVG(p)
        with open(f'{out_prefix}_p{p}.svg', 'w', encoding='utf-8') as o:
            o.write(svg)

for n in [BASIS, BASIS+2, BASIS+4]:
    render_verovio(f'cast_{n}.musicxml', f'cast_{n}')

---
---


# Puesta en común

## Limpieza de los musicxml:



### Las funciones:

In [5]:
from pathlib import Path
from music21 import converter, layout, stream

def limpiar_y_guardar_musicxml(ruta_entrada: str, carpeta_salida: str) -> str:
    """
    Limpia saltos de sistema/página del MusicXML y guarda <nombre>_limpiado.musicxml en carpeta_salida.
    Devuelve la ruta del archivo generado.
    """
    ruta_entrada = Path(ruta_entrada)
    carpeta_salida = Path(carpeta_salida)
    carpeta_salida.mkdir(parents=True, exist_ok=True)

    # Cargar
    sc = converter.parse(str(ruta_entrada))

    # Elimina TODOS los SystemLayout/PageLayout (saltos importados)
    for p in sc.parts:
        for m in p.recurse().getElementsByClass(stream.Measure):
            # System breaks
            for sl in list(m.getElementsByClass(layout.SystemLayout)):
                m.remove(sl)
            # Page breaks
            for pl in list(m.getElementsByClass(layout.PageLayout)):
                m.remove(pl)

    # Limpieza adicional a nivel de score
    for sl in list(sc.recurse().getElementsByClass(layout.SystemLayout)):
        sc.remove(sl)
    for pl in list(sc.recurse().getElementsByClass(layout.PageLayout)):
        sc.remove(pl)

    # Construir nombre de salida: <nombre_original>_limpiado.musicxml
    nombre_base = ruta_entrada.stem
    ruta_salida = carpeta_salida / f"{nombre_base}_limpiado.musicxml"

    # Guardar donde tú decidas
    sc.write('musicxml', fp=str(ruta_salida))

    return str(ruta_salida)


def limpiar_carpeta_musicxml(carpeta_entrada: str, carpeta_salida: str):
    """
    Aplica limpiar_y_guardar_musicxml() a todos los MusicXML de una carpeta.
    """
    carpeta_entrada = Path(carpeta_entrada)
    carpeta_salida = Path(carpeta_salida)
    carpeta_salida.mkdir(parents=True, exist_ok=True)

    # Extensiones típicas de MusicXML
    EXT = {".xml", ".musicxml", ".mxl"}

    rutas = sorted([p for p in carpeta_entrada.iterdir() if p.suffix.lower() in EXT])

    resultados = []
    for ruta in rutas:
        salida = limpiar_y_guardar_musicxml(str(ruta), str(carpeta_salida))
        resultados.append(salida)

    return resultados

### Uso:

In [4]:
from funciones import *

# Elimina los saltos de pentagrama y sistema del musicxml de UN fichero
limpiar_y_guardar_musicxml("cancion entera.musicxml", "res")

# Limpia una carpeta entera
limpiar_carpeta_musicxml("/content/", "res2")

['res2/cancion entera_limpiado.musicxml']

## Re-estructuración de los saltos de sistema del musicxml:

### Funciones:

In [ ]:
from music21 import layout

sc.definesExplicitSystemBreaks = True  # ayuda a que los lectores respeten <print new-system> [3](https://music21.org/music21docs/moduleReference/moduleLayout.html)

# Referencia de compases con la primera parte
measures = list(sc.parts[0].getElementsByClass(stream.Measure))

def imponer_compases_por_sistema(n, score, measures_ref):
    # Limpia posibles SystemLayout anteriores (por si re-ejecutas)
    for p in score.parts:
        for m in p.recurse().getElementsByClass(stream.Measure):
            for sl in list(m.getElementsByClass(layout.SystemLayout)):
                m.remove(sl)
    # Inserta saltos cada n compases (comienza nuevo sistema en 1+n, 1+2n, ...)
    for i, m in enumerate(measures_ref, start=1):
        if i != 1 and (i-1) % n == 0:
            m.insert(0, layout.SystemLayout(isNew=True))

# “Normal” (p.ej., 4 por sistema), “+2” y “+4”:
BASIS = 4
for extra in [0, 2, 4]:
    imponer_compases_por_sistema(BASIS+extra, sc, measures)
    sc.write('musicxml', fp=f'cast_{BASIS+extra}.musicxml')